In [8]:
# Obtain data
import pandas as pd
import io
import shap
#df_data = pd.read_csv(io.StringIO(data), header = 0)
#df_data = pd.read_csv('HousePrices_HalfMil.csv', header = 0)
df_data = pd.read_csv('train.csv', header = 0, nrows=300)
# Get predictor and target
x = df_data.drop("Prices",axis=1)
y_true = df_data["Prices"]
x=x.fillna(value=0)
y_true = y_true.fillna(value=0)
# Train regression
from sklearn.linear_model import Lasso
import numpy as np
#alpha_list = [0.01,0.1,1,2,5,10]
alpha_list = [1]
model_list=[]
rmse_list = []
r2_list = []
for alpha in alpha_list:
    lm = Lasso(alpha)
    lm.fit(x, y_true)
    model_list.append(lm)
    #model quality
    y_pred = lm.predict(x)
    mse = np.mean((y_pred - y_true)**2)
    rmse = np.sqrt(mse)
    r2 = lm.score(x,y_true)
    rmse_list.append(rmse)
    r2_list.append(r2)
# to send metrics to the Submit Metrics operator, create a Python dictionary of key-value pairs
metrics_dict={}
for i in range(len(alpha_list)):
    metrics_dict.update({(''.join(['alpha',str(i)])):str(alpha_list[i])})
    metrics_dict.update({(''.join(['RMSE',str(i)])):str(rmse_list[i])})
    metrics_dict.update({(''.join(['RSquare',str(i)])):str(r2_list[i])})
index =  r2_list.index(max(r2_list))
lm= model_list[index]
print(metrics_dict)


{'alpha0': '1', 'RMSE0': '5.119238042690786', 'RSquare0': '0.9999998329813352'}


In [9]:
feature_names = x.columns.tolist()
X_train = shap.kmeans(x, 10)
clf = lm
clf_fn = lm.predict
y_train = y_true
target_names_list =[]

In [10]:
import os
import json
import sys
sys.path.append('../../../')
from xai.compiler.base import Configuration, Controller

In [11]:
json_config = 'shap-tabular-regressor-feature-importance.json'
with open(json_config) as file:
    config = json.load(file)
config

{'name': 'Report for Housing Prices Half Mil Feature Importance Ranking',
 'overview': True,
 'content_table': True,
 'contents': [{'title': 'Feature Importance Ranking with Housing Prices data-set',
   'desc': 'This section provides the Feature Importance of model',
   'sections': [{'title': 'Feature Importance Analysis',
     'desc': 'This section provides the analysis on feature',
     'component': {'_comment': 'refer to document section xxxx',
      'class': 'FeatureImportanceRanking',
      'attr': {'trained_model': 'var:clf',
       'train_data': 'var:X_train',
       'feature_names': 'var:feature_names',
       'method': 'shap'}}}]}],
 'writers': [{'class': 'Pdf',
   'attr': {'name': 'housingprices-regression-feature-importance-report'}}]}

In [12]:
controller = Controller(config=Configuration(config, locals()))

In [13]:
controller.render()

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


AssertionError: Unknown instance type: <class 'shap.common.DenseData'>